# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
import time
import joblib
import numpy as np
import pandas as pd
import sqlalchemy as db
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to /Users/alexcheng/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/alexcheng/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
from sklearn.svm import SVC
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [4]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', engine)
df.head(3)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X = df.message.values
Y = df.drop(['id', 'message', 'original', 'genre'], axis=1).values

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidif', TfidfTransformer()),
    ('multiclf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [9]:
# Timing how long to train using pipeline
start = time.time()

pipeline.fit(X_train, y_train)

end = time.time()
print("Time elapsed:", (end - start) / 60, "minutes")

Time elapsed: 5.471842217445373 minutes


In [10]:
y_preds = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
accuracy = (y_preds == y_test).mean()
print("Accuracy:", accuracy)

Accuracy: 0.9467966975214458


In [12]:
target_names = list(df.drop(['id', 'message', 'original', 'genre'], axis=1))
#target_names

In [13]:
for i in range(len(target_names)):
    print("Column:", target_names[i])
    print(classification_report(y_test[:,i], y_preds[:,i]))
    print("----------------------------------------------------")

Column: related
              precision    recall  f1-score   support

           0       0.74      0.26      0.38      1515
           1       0.81      0.97      0.88      4991
           2       0.42      0.23      0.30        48

    accuracy                           0.80      6554
   macro avg       0.66      0.49      0.52      6554
weighted avg       0.79      0.80      0.76      6554

----------------------------------------------------
Column: request
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      5450
           1       0.91      0.44      0.60      1104

    accuracy                           0.90      6554
   macro avg       0.90      0.72      0.77      6554
weighted avg       0.90      0.90      0.88      6554

----------------------------------------------------
Column: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6527
           1       0.00      0.

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6420
           1       1.00      0.03      0.06       134

    accuracy                           0.98      6554
   macro avg       0.99      0.51      0.52      6554
weighted avg       0.98      0.98      0.97      6554

----------------------------------------------------
Column: tools
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6515
           1       0.00      0.00      0.00        39

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554

----------------------------------------------------
Column: hospitals
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6491
           1       0.00      0.00      0.00        63

    accuracy                           0.99 

### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
#print(pipeline.get_params().keys())

In [15]:
parameters = {
    'vect__ngram_range': [(1,1), (1,2)],
    'multiclf__estimator__n_estimators': [50, 100, 150],
}

cv = GridSearchCV(estimator=pipeline, 
                  param_grid=parameters, 
                  cv=5, n_jobs=-1)

In [16]:
# Timing how long to train using GridSearch
start = time.time()

cv.fit(X_train, y_train)

end = time.time()
print("Time elapsed:", (end - start) / 60, "minutes")

Time elapsed: 73.62232101758322 minutes


In [17]:
y_preds_cv = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [18]:
accuracy = (y_preds_cv == y_test).mean()
print("Accuracy:", accuracy)

Accuracy: 0.9476231648187705


In [19]:
for i in range(len(target_names)):
    print("Column:", target_names[i])
    print(classification_report(y_test[:,i], y_preds_cv[:,i]))
    print("----------------------------------------------------")

Column: related
              precision    recall  f1-score   support

           0       0.72      0.26      0.38      1515
           1       0.81      0.97      0.88      4991
           2       0.42      0.21      0.28        48

    accuracy                           0.80      6554
   macro avg       0.65      0.48      0.51      6554
weighted avg       0.78      0.80      0.76      6554

----------------------------------------------------
Column: request
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      5450
           1       0.89      0.47      0.62      1104

    accuracy                           0.90      6554
   macro avg       0.90      0.73      0.78      6554
weighted avg       0.90      0.90      0.89      6554

----------------------------------------------------
Column: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6527
           1       0.00      0.

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6420
           1       1.00      0.01      0.03       134

    accuracy                           0.98      6554
   macro avg       0.99      0.51      0.51      6554
weighted avg       0.98      0.98      0.97      6554

----------------------------------------------------
Column: tools
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6515
           1       0.00      0.00      0.00        39

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554

----------------------------------------------------
Column: hospitals
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6491
           1       1.00      0.02      0.03        63

    accuracy                           0.99 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [20]:
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidif', TfidfTransformer()),
    ('svc_clf', MultiOutputClassifier(OneVsRestClassifier(SVC())))
])

In [21]:
# Timing how long to train using pipeline
start = time.time()

pipeline2.fit(X_train, y_train)

end = time.time()
print("Time elapsed:", (end - start) / 60, "minutes")

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/multiclass.py:74: UserWarning: Label not 0 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %


Time elapsed: 48.287166118621826 minutes


In [22]:
y_preds_SVC = pipeline2.predict(X_train)

In [25]:
# accuracy = (y_preds_SVC == y_test).mean()
# print("Accuracy:", accuracy)

### 9. Export your model as a pickle file

In [26]:
# Save the GridSearch model
joblib.dump(cv, '../models/classifier.pkl') 

['../models/classifier.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.